# Import

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns
import matplotlib.pyplot as plt
import sqlite3

# Read SQL function

In [2]:
def map_tech_to_mode(tech):
    class_mapping = {
    'T_HDV_AJ': 'Air jet',
    'T_HDV_B': 'Bus',
    'T_HDV_R': 'Rail',
    'T_HDV_T': 'HD Truck',
    'T_HDV_W': 'Marine vessel',
    'T_MDV_T': 'MD Truck',
    'T_LDV_C_': 'LD Car',
    'T_LDV_LT': 'LD Truck',
    'T_LDV_M': 'Motorcycle',
    'T_IMP_': 'Fuel use',
    'H2_COMP_100_700': 'Hydrogen use',
    'CHRG' : 'Charger'
    }
    for prefix, class_name in class_mapping.items():
        if tech.startswith(prefix):
            return class_name
    return 'Other'

def map_tech_to_fuel(tech):
    carrier_mapping = {
        'BEV': 'Battery electric',
        'GSL': 'Gasoline',
        'DSL': 'Diesel',
        'CNG': 'Compressed NG',
        'LNG': 'Liquified NG',
        'NG' : 'Natural Gas',
        'JTF': 'Jet Fuel',
        'JFL': 'Jet Fuel',
        'SPK': 'Synth. Jet Fuel',
        'HFO': 'Heavy Fuel Oil',
        'MDO': 'Marine Diesel Oil',
        'ELC': 'Electricity',
        'ETH': 'Ethanol',	
        'RDSL': 'Ren. Diesel'
    }
    # Order matters 
    if 'PHEV35' in tech:
        return 'PHEV (35-mile AER)'
    if 'PHEV50' in tech:
        return 'PHEV (50-mile AER)'
    if 'PHEV' in tech:
        return 'Plug-in hybrid'
    if 'BEV150' in tech:
        return 'BEV (150-mile AER)'
    if 'BEV200' in tech:
        return 'BEV (200-mile AER)'
    if 'BEV300' in tech:
        return 'BEV (300-mile AER)'
    if 'BEV400' in tech:
        return 'BEV (400-mile AER)'
    if 'FC' in tech:
        return 'Fuel-cell electric'
    if 'HEV' in tech:
        return 'Hybrid'
    if 'H2' in tech:
        return 'Hydrogen'
    if 'BEV_CHRG' in tech:
        return 'LD BEV charger'
    if 'CHRG' in tech:
        return 'Other charger'
    for prefix, carrier in carrier_mapping.items():
        if prefix in tech:
            return carrier 
    return 'Other'

def map_tech_to_plant(tech):
    plant_mapping = {
    'E_BIO': 'Biomass',
    'E_NG': 'Natural Gas',
    'E_HYD': 'Hydro',
    'E_SOL_PV': 'Solar PV',
    'E_WND': 'Wind',
    'E_NUC': 'Nuclear',
    'E_BAT': 'Batteries',
    'E_PUMP': 'Pumped storage',
    'E_IMP_': 'Fuel use',
    }
    for prefix, plant_name in plant_mapping.items():
        if tech.startswith(prefix):
            return plant_name
    return 'Other'

In [3]:
def load_transport_newcapacity(db_variant='vanilla4', path='C:/Users/rashi/ESM_databases/temoa/data_files/'):
    db_file = path + f'canoe_on_12d_{db_variant}.sqlite'
    conn = sqlite3.connect(db_file)

    # filter db tables
    query_built_capacity = f"SELECT * FROM OutputBuiltCapacity WHERE sector = 'Transport'"
    df = pd.read_sql_query(query_built_capacity, conn).drop(columns=['region', 'sector'])
    conn.close()

    # map technology to mode and fuel and filter for road modes 
    df['mode'] = df['tech'].apply(map_tech_to_mode)
    df = df[df['mode'].isin(['LD Car', 'LD Truck', 'MD Truck', 'HD Truck'])]
    df['fuel'] = df['tech'].apply(map_tech_to_fuel)

    # aggregate new capacities and filter zeroes
    df = df.groupby(['mode', 'fuel', 'vintage'], as_index=False).sum('capacity').rename(columns={'capacity':'value', 'vintage':'period'})
    df['value'] = df['value'].round(3)
    df = df[df['value'] > 0]

    # sort and add labels
    df = df.sort_values(by=['mode', 'fuel', 'period'])
    df['variable'] = 'transport_newcap'
    df['scenario'] = db_variant
    return df

In [4]:
load_transport_newcapacity()

,mode,fuel,period,value,variable,scenario
1,HD Truck,Battery electric,2050,7.140,transport_newcap,vanilla4
8,HD Truck,Fuel-cell electric,2030,25.405,transport_newcap,vanilla4
9,HD Truck,Fuel-cell electric,2035,42.500,transport_newcap,vanilla4
10,HD Truck,Fuel-cell electric,2040,72.073,transport_newcap,vanilla4
11,HD Truck,Fuel-cell electric,2045,27.598,transport_newcap,vanilla4
...,...,...,...,...,...,...
183,MD Truck,Fuel-cell electric,2030,113.745,transport_newcap,vanilla4
184,MD Truck,Fuel-cell electric,2035,182.177,transport_newcap,vanilla4
185,MD Truck,Fuel-cell electric,2040,55.021,transport_newcap,vanilla4
186,MD Truck,Fuel-cell electric,2045,8.696,transport_newcap,vanilla4


In [5]:
def load_transport_energy(db_variant='vanilla4', path='C:/Users/rashi/ESM_databases/temoa/data_files/'):
    db_file = path + f'canoe_on_12d_{db_variant}.sqlite'
    conn = sqlite3.connect(db_file)

    df = pd.read_sql_query(
        "SELECT period, tech, flow FROM OutputFlowOut WHERE sector = 'Transport'",
        conn
    )
    conn.close()

    # map technology to mode and fuel and filter out unwanted technologies
    df['mode'] = df['tech'].apply(map_tech_to_mode)
    df = df[df['mode'].isin(['Fuel use', 'Hydrogen use'])]
    df['fuel'] = df['tech'].apply(map_tech_to_fuel)
    df = df[df['fuel'] != 'Natural Gas']    # natural gas is used for hydrogen production

    # aggregate and filter zeroes
    df = df.groupby(['mode', 'fuel', 'period'], as_index=False).sum('flow').rename(columns={'flow':'value'})
    df['value'] = df['value'].round(3)
    df = df[df['value'] > 0]

    # sort and add labels
    df = df.sort_values(by=['mode', 'fuel', 'period'])
    df['variable'] = 'transport_energy'
    df['scenario'] = db_variant
    return df

In [6]:
load_transport_energy().fuel.unique()

array(['Compressed NG', 'Diesel', 'Electricity', 'Ethanol', 'Gasoline',
       'Heavy Fuel Oil', 'Jet Fuel', 'Liquified NG', 'Marine Diesel Oil',
       'Hydrogen'], dtype=object)

In [7]:
def load_electricity_newcapacity(db_variant='vanilla4', path='C:/Users/rashi/ESM_databases/temoa/data_files/'):
    db_file = path + f'canoe_on_12d_{db_variant}.sqlite'
    conn = sqlite3.connect(db_file)

    # filter db tables
    query_built_capacity = f"SELECT * FROM OutputBuiltCapacity WHERE sector = 'electricity' and tech like '%NEW%'"
    df = pd.read_sql_query(query_built_capacity, conn).drop(columns=['region', 'sector'])
    conn.close()

    # map technology to mode and fuel and filter out unwanted technologies
    df['mode'] = df['tech'].apply(map_tech_to_plant)
    df = df[df['mode'] != 'Fuel use']
    df['fuel'] = df['tech'].apply(map_tech_to_fuel)

    # aggregate and filter zeroes
    df = df.groupby(['mode', 'fuel', 'vintage'], as_index=False).sum('capacity').rename(columns={'capacity':'value', 'vintage':'period'})
    df['value'] = df['value'].round(5)
    df = df[df['value'] > 0]

    # sort and add labels
    df = df.sort_values(by=['mode', 'fuel', 'period'])
    df['variable'] = 'electricity_newcap'
    df['scenario'] = db_variant
    return df

In [8]:
load_electricity_newcapacity()

,mode,fuel,period,value,variable,scenario
0,Batteries,Other,2021,0.00023,electricity_newcap,vanilla4
1,Batteries,Other,2025,0.00025,electricity_newcap,vanilla4
2,Batteries,Other,2030,0.00042,electricity_newcap,vanilla4
3,Batteries,Other,2035,0.00057,electricity_newcap,vanilla4
4,Batteries,Other,2040,0.00077,electricity_newcap,vanilla4
5,Batteries,Other,2045,0.00135,electricity_newcap,vanilla4
6,Batteries,Other,2050,0.00369,electricity_newcap,vanilla4
7,Natural Gas,Natural Gas,2021,9.44004,electricity_newcap,vanilla4
8,Natural Gas,Natural Gas,2025,4.62195,electricity_newcap,vanilla4
9,Natural Gas,Natural Gas,2030,7.74608,electricity_newcap,vanilla4


In [9]:
def load_emissions(db_variant='vanilla4', path='C:/Users/rashi/ESM_databases/temoa/data_files/'):
    db_file = path + f'canoe_on_12d_{db_variant}.sqlite'
    conn = sqlite3.connect(db_file)

    df = pd.read_sql_query(
        # "SELECT period, tech, emission FROM OutputEmission WHERE sector = 'Transport' and emis_comm = 'co2e'",
        "SELECT sector, period, sum(emission) FROM OutputEmission WHERE emis_comm = 'co2e' GROUP BY period, sector",
        conn
    )
    conn.close()

    # map technology to mode and fuel and filter out unwanted technologies
    # df['mode'] = df['tech'].apply(map_tech_to_mode)
    # df = df[df['mode'] == 'Fuel use']
    # df['fuel'] = df['tech'].apply(map_tech_to_fuel)
    df['fuel'] = None

    # aggregate and filter zeroes
    df = df.rename(columns={'sum(emission)':'value', 'sector':'mode'})
    df['value'] = df['value'] / 1000   # convert to MtCO2e
    df['value'] = df['value'].round(3)

    # sort and add labels
    df = df.sort_values(by=['mode', 'period'])
    df['variable'] = 'total_emissions'
    df['scenario'] = db_variant
    return df

In [10]:
load_emissions()

,mode,period,value,fuel,variable,scenario
0,Transport,2021,68.185,None,total_emissions,vanilla4
5,Transport,2025,77.333,None,total_emissions,vanilla4
10,Transport,2030,59.860,None,total_emissions,vanilla4
15,Transport,2035,37.506,None,total_emissions,vanilla4
20,Transport,2040,31.033,None,total_emissions,vanilla4
25,Transport,2045,31.431,None,total_emissions,vanilla4
30,Transport,2050,31.478,None,total_emissions,vanilla4
1,commercial,2021,18.444,None,total_emissions,vanilla4
6,commercial,2025,18.531,None,total_emissions,vanilla4
11,commercial,2030,18.560,None,total_emissions,vanilla4


In [11]:
def load_dis_costs(db_variant='vanilla4', path='C:/Users/rashi/ESM_databases/temoa/data_files/'):
    db_file = path + f'canoe_on_12d_{db_variant}.sqlite'
    conn = sqlite3.connect(db_file)

    df = pd.read_sql_query(
        "SELECT period, sum(d_invest), sum(d_fixed), sum(d_var) FROM OutputCost GROUP BY period",
        conn
    )
    conn.close()

    # map technology to mode and fuel and filter out unwanted technologies
    df['mode'] = None
    df['fuel'] = None

    # aggregate and filter zeroes
    df['cost_total'] = df[['sum(d_invest)','sum(d_fixed)','sum(d_var)']].sum(axis=1) / 1000  # convert to billion CAD
    df = df.rename(columns={'cost_total':'value'})
    df['value'] = df['value'].round(3)

    # sort and add labels
    df = df.sort_values(by=['period'])
    df['variable'] = 'total_transport_costs'
    df['scenario'] = db_variant
    return df[['mode', 'fuel', 'period', 'value', 'variable', 'scenario']]

In [12]:
load_dis_costs()

,mode,fuel,period,value,variable,scenario
0,None,None,2021,291.492,total_transport_costs,vanilla4
1,None,None,2025,404.498,total_transport_costs,vanilla4
2,None,None,2030,333.983,total_transport_costs,vanilla4
3,None,None,2035,308.917,total_transport_costs,vanilla4
4,None,None,2040,258.093,total_transport_costs,vanilla4
5,None,None,2045,194.268,total_transport_costs,vanilla4
6,None,None,2050,152.237,total_transport_costs,vanilla4


In [13]:
load_transport_newcapacity(db_variant='baseline').fuel.unique()

array(['Battery electric', 'Diesel', 'Fuel-cell electric',
       'BEV (150-mile AER)', 'BEV (200-mile AER)', 'BEV (300-mile AER)',
       'BEV (400-mile AER)', 'Gasoline', 'Hybrid', 'PHEV (35-mile AER)',
       'PHEV (50-mile AER)'], dtype=object)

In [14]:
def build_master_df(db_variants, path='C:/Users/rashi/ESM_databases/temoa/data_files/'):
    all_parts = []

    for variant in db_variants:
        # --- 1) transport_newcapacity: BEV / FCEV / Hybrid ---
        df_tnc = load_transport_newcapacity(db_variant=variant, path=path)
        # classify into the three buckets
        def cat_tnc(fuel):
            if 'BEV'    in fuel or 'Battery' in fuel: return 'BEV'
            if fuel == 'Fuel-cell electric':           return 'FCEV'
            if any(k in fuel for k in ['PHEV','Hybrid', 'Plug-in']): return 'Hybrids'
            return None

        df_tnc['category'] = df_tnc['fuel'].map(cat_tnc)
        df_tnc = (
            df_tnc
            .dropna(subset=['category'])
            .groupby(['scenario','period','variable','category'], as_index=False)
            ['value'].sum()
        )
        all_parts.append(df_tnc)

        # --- 2) transport_energy: Electricity vs Chemical ---
        df_te = load_transport_energy(db_variant=variant, path=path)
        df_te['category'] = df_te['fuel'].apply(lambda f: 'Electricity' if f=='Electricity' else 'Chemical fuels')
        df_te = (
            df_te
            .groupby(['scenario','period','variable','category'], as_index=False)
            ['value'].sum()
        )
        all_parts.append(df_te)

        # --- 3) electricity_newcapacity: VRE vs Total ---
        df_enc = load_electricity_newcapacity(db_variant=variant, path=path)
        # VRE = Solar PV or Wind
        vre = df_enc[df_enc['mode'].isin(['Solar PV','Wind'])].groupby(
            ['scenario','period'], as_index=False
        )['value'].sum().assign(variable='electricity_newcap', category='VRE gen')
        tot = df_enc.groupby(
            ['scenario','period'], as_index=False
        )['value'].sum().assign(variable='electricity_newcap', category='Total gen')
        all_parts += [tot, vre]

        # --- 4) cumulative emissions ---
        df_em = load_emissions(db_variant=variant, path=path)
        df_em = (
            df_em
            .groupby(['scenario','period'], as_index=False)['value'].sum()
            .sort_values(['scenario','period'])
        )
        df_em['value'] = df_em.groupby('scenario')['value'].cumsum()
        df_em['variable'] = 'cumulative_emissions'
        df_em['category'] = None
        all_parts.append(df_em[['scenario','period','variable','category','value']])

        # --- 5) cumulative costs ---
        df_cost = load_dis_costs(db_variant=variant, path=path)
        df_cost = (
            df_cost
            .groupby(['scenario','period'], as_index=False)['value'].sum()
            .sort_values(['scenario','period'])
        )
        df_cost['value'] = df_cost.groupby('scenario')['value'].cumsum()
        df_cost['variable'] = 'cumulative_costs'
        df_cost['category'] = None
        all_parts.append(df_cost[['scenario','period','variable','category','value']])

    master_df = pd.concat(all_parts, ignore_index=True)
    return master_df

In [15]:
df_scenarios = build_master_df(
    db_variants=[
        'baseline', 
        'vanilla4',
        'lowgrowth', 
        'medgrowth', 
        'medgrowth_aeo',
        'evgrowth', 
        'highgrowth', 
        'baseline_life_3',
        'baseline_life_7', 
        'medgrowth_life_7', 
        'baseline_tailpipe', 
        'embodied',
        'embodied_medgrowth',
        'embodied_life_7',
        'embodied_medgrowth_life_7',
        'baseline_nhts',
        'baseline_nhts_fixed',
        'baseline_car_shift',
        'baseline_truck_usage',
        'baseline_usage_trend',
        ],
)

df_scenarios

,scenario,period,variable,category,value
0,baseline,2021,transport_newcap,BEV,43.560
1,baseline,2021,transport_newcap,Hybrids,88.270
2,baseline,2025,transport_newcap,BEV,86.987
3,baseline,2025,transport_newcap,FCEV,0.002
4,baseline,2025,transport_newcap,Hybrids,3657.570
...,...,...,...,...,...
1216,baseline_usage_trend,2030,cumulative_costs,None,1185.722
1217,baseline_usage_trend,2035,cumulative_costs,None,1554.179
1218,baseline_usage_trend,2040,cumulative_costs,None,1860.257
1219,baseline_usage_trend,2045,cumulative_costs,None,2080.661


In [16]:
df_scenarios.to_csv('scenario_results.csv', index=False)

In [17]:
import pandas as pd

def prepare_for_pivot(
    master_df,
    baseline_scenario,
    abs_diff_categories=['FCEV', 'Hybrid']  # categories for which you want absolute diffs
):
    """
    master_df             : raw all-scenarios table
    baseline_scenario     : e.g. "vanilla4"
    abs_diff_categories   : list of category-names (e.g. ['FCEV','Hybrid'])
                            for which you want absolute diffs instead of percent

    Returns a df with:
      - all baseline rows,
      - rows named diff_<variable> containing either
         * absolute change (scenario – baseline) if category∈abs_diff_categories,
         * percent change ((scenario–baseline)/baseline) otherwise.
    """
    if abs_diff_categories is None:
        abs_diff_categories = []

    # 1) split baseline / alt
    df_base = master_df[master_df['scenario'] == baseline_scenario].copy()
    df_alt  = master_df[master_df['scenario'] != baseline_scenario].copy()

    # 2) merge to get baseline values side by side
    df_cmp = df_alt.merge(
        df_base[['period','variable','category','value']],
        on=['period','variable','category'],
        suffixes=('','_base')
    )

    # 3) compute diff_value differently for the special categories
    mask = df_cmp['category'].isin(abs_diff_categories)

    # absolute difference: (value – value_base)
    df_cmp.loc[mask, 'diff_value'] = (
        df_cmp.loc[mask, 'value'] - df_cmp.loc[mask, 'value_base']
    )

    # percent difference: (value – value_base) / value_base
    df_cmp.loc[~mask, 'diff_value'] = (
        (df_cmp.loc[~mask, 'value'] - df_cmp.loc[~mask, 'value_base'])
        / df_cmp.loc[~mask, 'value_base']
    )

    # 4) round & label
    df_cmp['diff_value'] = df_cmp['diff_value'].round(3)
    df_diff = (
        df_cmp
        .assign(
            variable = lambda d: 'diff_' + d['variable'],
            value    = lambda d: d['diff_value'],
            scenario = lambda d: d['scenario']
        )
        [['scenario','period','variable','category','value']]
    )

    # 5) stitch baseline + diffs
    pivot_ready = pd.concat([df_base, df_diff], ignore_index=True)
    return pivot_ready


In [18]:
pivot_ready = prepare_for_pivot(df_scenarios, 'baseline')

pivot_ready[(pivot_ready['category'] == 'Hybrid') & (pivot_ready['period'] == 2050)]

,scenario,period,variable,category,value


In [19]:
def make_scenario_pivot(
    master_df,
    baseline_scenario='baseline',
    periods=[2035, 2050]
):
    """
    master_df           : the raw all-scenarios table
    baseline_scenario   : the name of your baseline (e.g. "vanilla4")
    periods             : list of ints or strings, e.g. [2035, 2050].
                          If None, all periods are kept.

    Returns a DataFrame indexed by 'scenario', with MultiIndex columns:
      level 0 = variable (with category appended if present),
      level 1 = period.
    Missing values are replaced with 0.
    """

    # 1) get baseline + diff rows
    df = prepare_for_pivot(master_df, baseline_scenario).copy()

    # 2) restrict to only the chosen periods
    if periods is not None:
        df = df[df['period'].isin(periods)]

    # 3) strip off the 'diff_' prefix so all scenarios line up
    df['variable'] = df['variable'].str.replace(r'^diff_', '', regex=True)

    # 4) build a combined 'variable[_category]' label
    def _vc_label(r):
        v = r['variable']
        if pd.notna(r['category']):
            v = f"{v}_{r['category']}"
        return v

    df['var_label'] = df.apply(_vc_label, axis=1)

    # 5) remember the order in which scenarios and var_labels first appeared
    scenario_order = master_df['scenario'].drop_duplicates().tolist()
    var_order      = df['var_label'].drop_duplicates().tolist()
    period_order   = list(periods) if periods is not None else df['period'].drop_duplicates().tolist()

    # 6) do the pivot
    pivot = df.pivot(
        index   = 'scenario',
        columns = ['var_label', 'period'],
        values  = 'value'
    )

    # 7) reindex rows & columns to preserve that original order
    #    rows = baseline + the rest in the order from master_df
    pivot = pivot.reindex(index=scenario_order)

    #    columns: for each var_label in var_order, each period in period_order
    col_index = pd.MultiIndex.from_tuples(
        [(v, p) for v in var_order for p in period_order],
        names=['variable','period']
    )
    pivot = pivot.reindex(columns=col_index)

    # 8) replace any NaN with 0
    pivot = pivot.fillna(0)

    # 9) tidy up names
    pivot.index.name   = None

    return pivot


In [20]:
pivot_df = make_scenario_pivot(
    master_df=df_scenarios,
    baseline_scenario='baseline',
    periods=[2035, 2050]
)

pivot_df

variable                  transport_newcap_BEV            \
period                                    2035      2050   
baseline                              5592.435  6397.330   
vanilla4                                -0.022    -0.007   
lowgrowth                               -0.878    -0.117   
medgrowth                               -0.582    -0.011   
medgrowth_aeo                           -0.583    -0.487   
evgrowth                                -0.056    -0.001   
highgrowth                               0.005    -0.000   
baseline_life_3                         -0.171    -0.102   
baseline_life_7                         -0.183    -0.139   
medgrowth_life_7                        -0.533    -0.156   
baseline_tailpipe                        0.000     0.000   
embodied                                -0.000    -0.000   
embodied_medgrowth                      -0.582    -0.011   
embodied_life_7                         -0.186    -0.141   
embodied_medgrowth_life_7               -0.533    -0.159   
baseline_nhts                            0.005     0.000   
baseline_nhts_fixed                     -0.000    -0.000   
baseline_car_shift                      -0.031    -0.046   
baseline_truck_usage                     0.019     0.021   
baseline_usage_trend                     0.160     0.347   

variable                  transport_newcap_FCEV      transport_newcap_Hybrids  \
period                                     2035 2050                     2035   
baseline                                  0.004  0.0                  109.942   
vanilla4                                224.673  0.0                   -0.866   
lowgrowth                                56.909  0.0                   13.375   
medgrowth                               126.634  0.0                   23.512   
medgrowth_aeo                             6.537  0.0                   24.396   
evgrowth                                 39.292  0.0                    1.484   
highgrowth                                0.571  0.0                   -0.353   
baseline_life_3                          -0.001  0.0                    0.027   
baseline_life_7                           0.000  0.0                    0.117   
medgrowth_life_7                         78.505  0.0                   14.274   
baseline_tailpipe                        -0.003  0.0                    0.000   
embodied                                  0.002  0.0                    0.006   
embodied_medgrowth                      126.607  0.0                   23.512   
embodied_life_7                          -0.002  0.0                    0.235   
embodied_medgrowth_life_7                78.504  0.0                   14.274   
baseline_nhts                            -0.001  0.0                   -0.233   
baseline_nhts_fixed                      -0.003  0.0                    0.001   
baseline_car_shift                       -0.003  0.0                    0.089   
baseline_truck_usage                      0.000  0.0                    0.022   
baseline_usage_trend                      0.000  0.0                   -0.120   

variable                            transport_energy_Chemical fuels           \
period                         2050                            2035     2050   
baseline                      1.827                         410.600  260.420   
vanilla4                      1.537                           0.081    0.474   
lowgrowth                   284.592                           1.232    1.155   
medgrowth                    14.581                           0.877    0.587   
medgrowth_aeo              1682.758                           1.001    0.996   
evgrowth                      3.215                           0.415    0.123   
highgrowth                    0.066                           0.175    0.023   
baseline_life_3              24.060                           0.254    0.038   
baseline_life_7              60.014                           0.252    0.104   
medgrowth_life_7

In [21]:
pivot_df.to_clipboard()